In [1]:
from pyspark.sql import SparkSession
from sklearn import datasets
import time

#1.2 Initialize a SparkSession
spark = SparkSession.builder \
    .appName("SimModeExample") \
    .master("local[*]") \
    .getOrCreate()
sc = spark.sparkContext

In [2]:
#Printing Configs
for k, v in spark.sparkContext.getConf().getAll():
    print(f"{k} = {v}")

spark.app.submitTime = 1755099849606
spark.driver.extraJavaOptions = -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.executor.id = driver
spark.driver.host = 5cdbd42e068a
spark.app.id = 

In [3]:
sample_text = """Apache Spark is an open-source distributed general-purpose cluster-computing framework.
It provides an interface for programming entire clusters with implicit data parallelism and fault-tolerance.
Spark is designed to cover a wide range of workloads such as batch applications, iterative algorithms, interactive queries, and streaming."""

with open("sample_text.txt", "w") as f:
    f.write(sample_text)

In [4]:
#1.4 Create RDDs from lists or text files
text_rdd = sc.textFile("sample_text.txt")

In [5]:
#1.5	map(), filter(), flatMap() transformations
print(text_rdd.flatMap(lambda line: line.split(" ")).collect())
print(text_rdd.flatMap(lambda line: line.split(" ")).filter(lambda x: "a" in x).collect())

['Apache', 'Spark', 'is', 'an', 'open-source', 'distributed', 'general-purpose', 'cluster-computing', 'framework.', 'It', 'provides', 'an', 'interface', 'for', 'programming', 'entire', 'clusters', 'with', 'implicit', 'data', 'parallelism', 'and', 'fault-tolerance.', 'Spark', 'is', 'designed', 'to', 'cover', 'a', 'wide', 'range', 'of', 'workloads', 'such', 'as', 'batch', 'applications,', 'iterative', 'algorithms,', 'interactive', 'queries,', 'and', 'streaming.']
['Apache', 'Spark', 'an', 'general-purpose', 'framework.', 'an', 'interface', 'programming', 'data', 'parallelism', 'and', 'fault-tolerance.', 'Spark', 'a', 'range', 'workloads', 'as', 'batch', 'applications,', 'iterative', 'algorithms,', 'interactive', 'and', 'streaming.']


In [6]:
#1.6	collect(), count(), take() actions
print(text_rdd.flatMap(lambda line: line.split(" ")).filter(lambda x: "a" in x).count())
print(text_rdd.flatMap(lambda line: line.split(" ")).filter(lambda x: "a" in x).collect())
print(text_rdd.flatMap(lambda line: line.split(" ")).filter(lambda x: "a" in x).take(3))

24
['Apache', 'Spark', 'an', 'general-purpose', 'framework.', 'an', 'interface', 'programming', 'data', 'parallelism', 'and', 'fault-tolerance.', 'Spark', 'a', 'range', 'workloads', 'as', 'batch', 'applications,', 'iterative', 'algorithms,', 'interactive', 'and', 'streaming.']
['Apache', 'Spark', 'an']


In [7]:
#1.7	Create DataFrame from Python dictionary/list
data_frame_data = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 30},
    {"name": "Charlie", "surname:":"theron","age": 35}
]

df = spark.createDataFrame(data_frame_data)
df.show()

+---+-------+--------+
|age|   name|surname:|
+---+-------+--------+
| 25|  Alice|    NULL|
| 30|    Bob|    NULL|
| 35|Charlie|  theron|
+---+-------+--------+



In [39]:
df.collect()

[Row(Name='Alice', Subject='Math', Score=95),
 Row(Name='Bob', Subject='Math', Score=90),
 Row(Name='Charlie', Subject='Math', Score=90),
 Row(Name='David', Subject='Math', Score=85),
 Row(Name='Alice', Subject='Physics', Score=88),
 Row(Name='Bob', Subject='Physics', Score=92),
 Row(Name='Charlie', Subject='Physics', Score=85)]

In [8]:
#1.8	Create DataFrame from CSV/JSON/Parquet
csv_data = """id,name,age,salary
1,Alice,30,100000
2,Bob,,85000
3,Charlie,25,70000
4,David,35,
5,Eve,29,90000
"""

with open("sample_employees.csv", "w") as f:
    f.write(csv_data)

df = spark.read.csv("sample_employees.csv", header=True, inferSchema=True)
df.show()

+---+-------+----+------+
| id|   name| age|salary|
+---+-------+----+------+
|  1|  Alice|  30|100000|
|  2|    Bob|NULL| 85000|
|  3|Charlie|  25| 70000|
|  4|  David|  35|  NULL|
|  5|    Eve|  29| 90000|
+---+-------+----+------+



In [9]:
#1.9	Show schema and data (printSchema(), show())
print(df.printSchema())

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

None


In [44]:
df.schema

StructType([StructField('Name', StringType(), True), StructField('Subject', StringType(), True), StructField('Score', LongType(), True)])

In [10]:
#1.10	Select specific columns
df.select("name","age").show()

+-------+----+
|   name| age|
+-------+----+
|  Alice|  30|
|    Bob|NULL|
|Charlie|  25|
|  David|  35|
|    Eve|  29|
+-------+----+



In [11]:
#1.11	Filter rows using conditions
test_lambda_func = lambda x: x-2
df.select("name","age").filter(test_lambda_func(df.age)>25).show()

+-----+---+
| name|age|
+-----+---+
|Alice| 30|
|David| 35|
|  Eve| 29|
+-----+---+



In [46]:
#1.12	Rename columns
df = df.withColumnRenamed("name", "full_name")
df.printSchema()

new_column_names = ["user_id", "full_name","age", "salary net"]
df_renamed = df.toDF(*new_column_names)
df_renamed.printSchema()

root
 |-- full_name: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Score: long (nullable = true)



IllegalArgumentException: requirement failed: The number of columns doesn't match.
Old column names (3): full_name, Subject, Score
New column names (4): user_id, full_name, age, salary net

In [13]:
#1.13	Add new columns (withColumn)
df = df.withColumn("age2", df.age+2)
df.printSchema()
df.show()

root
 |-- id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- age2: integer (nullable = true)

+---+---------+----+------+----+
| id|full_name| age|salary|age2|
+---+---------+----+------+----+
|  1|    Alice|  30|100000|  32|
|  2|      Bob|NULL| 85000|NULL|
|  3|  Charlie|  25| 70000|  27|
|  4|    David|  35|  NULL|  37|
|  5|      Eve|  29| 90000|  31|
+---+---------+----+------+----+



In [14]:
#1.14	Drop columns
df = df.drop("age2")
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [15]:
#1.15	Register DataFrame as a SQL temporary view
#1.16	Run simple SELECT queries with spark.sql()

df.createOrReplaceTempView("people")
result = spark.sql("SELECT full_name, age FROM people WHERE age > 26")
result.show()

# createOrReplaceTempView: Visible only within the current Spark session
# createGlobalTempView   : Visible across multiple Spark sessions and accessed with the prefix global_temp.

+---------+---+
|full_name|age|
+---------+---+
|    Alice| 30|
|    David| 35|
|      Eve| 29|
+---------+---+



In [16]:
from pyspark.sql.functions import concat, upper, lower, trim, col, lit
#New Sample DF

data = [(" Alice ", "Smith","01-01-1996"), (" Bob ", "Brown","01-01-1996"), (" Cathy ", "Johnson","01-01-1996")]
columns = ["first_name", "last_name", "dob"]

df = spark.createDataFrame(data, columns)
df.show()

#2.1	String functions (concat, upper, lower, trim)
df = df.withColumn("first_name", trim(col("first_name")))
df = df.withColumn("last_name", trim(col("last_name")))
df = df.withColumn("Fullname", concat(col("first_name"),lit(" "),"last_name"))
df.show()

+----------+---------+----------+
|first_name|last_name|       dob|
+----------+---------+----------+
|    Alice |    Smith|01-01-1996|
|      Bob |    Brown|01-01-1996|
|    Cathy |  Johnson|01-01-1996|
+----------+---------+----------+

+----------+---------+----------+-------------+
|first_name|last_name|       dob|     Fullname|
+----------+---------+----------+-------------+
|     Alice|    Smith|01-01-1996|  Alice Smith|
|       Bob|    Brown|01-01-1996|    Bob Brown|
|     Cathy|  Johnson|01-01-1996|Cathy Johnson|
+----------+---------+----------+-------------+



In [17]:
from pyspark.sql.functions import to_date, col, datediff, current_date, floor
#2.2	Date/time functions (current_date, datediff, date_format)
df = df.withColumn("dob", to_date(col("dob"), format= "dd-mm-yyyy"))
df = df.withColumn("age", floor(datediff(current_date(), col("dob"))/365))
df.show()

+----------+---------+----------+-------------+---+
|first_name|last_name|       dob|     Fullname|age|
+----------+---------+----------+-------------+---+
|     Alice|    Smith|1996-01-01|  Alice Smith| 29|
|       Bob|    Brown|1996-01-01|    Bob Brown| 29|
|     Cathy|  Johnson|1996-01-01|Cathy Johnson| 29|
+----------+---------+----------+-------------+---+



In [18]:
from pyspark.sql.functions import to_date, col, when
#2.3	Conditional logic with when() and otherwise()
df.withColumn("age2",when(col("age") > 25, "y").otherwise("n")).show()

+----------+---------+----------+-------------+---+----+
|first_name|last_name|       dob|     Fullname|age|age2|
+----------+---------+----------+-------------+---+----+
|     Alice|    Smith|1996-01-01|  Alice Smith| 29|   y|
|       Bob|    Brown|1996-01-01|    Bob Brown| 29|   y|
|     Cathy|  Johnson|1996-01-01|Cathy Johnson| 29|   y|
+----------+---------+----------+-------------+---+----+



In [19]:
#2.4	groupBy() with aggregation functions (count, avg, sum, max, min)
df.groupBy("age").agg({"age":"count"}).show()

+---+----------+
|age|count(age)|
+---+----------+
| 29|         3|
+---+----------+



In [20]:
#2.5	Multiple aggregations in one statement
df.groupBy("age").agg({"age":"count","age":"avg"}).show()

+---+--------+
|age|avg(age)|
+---+--------+
| 29|    29.0|
+---+--------+



In [21]:
#2.6	orderBy() ascending/descending
df.orderBy(col("first_name").desc()).show()

+----------+---------+----------+-------------+---+
|first_name|last_name|       dob|     Fullname|age|
+----------+---------+----------+-------------+---+
|     Cathy|  Johnson|1996-01-01|Cathy Johnson| 29|
|       Bob|    Brown|1996-01-01|    Bob Brown| 29|
|     Alice|    Smith|1996-01-01|  Alice Smith| 29|
+----------+---------+----------+-------------+---+



In [22]:
#2.7	Multi-column ordering
df.orderBy([col("last_name").desc(),col("first_name").desc()]).show()

+----------+---------+----------+-------------+---+
|first_name|last_name|       dob|     Fullname|age|
+----------+---------+----------+-------------+---+
|     Alice|    Smith|1996-01-01|  Alice Smith| 29|
|     Cathy|  Johnson|1996-01-01|Cathy Johnson| 29|
|       Bob|    Brown|1996-01-01|    Bob Brown| 29|
+----------+---------+----------+-------------+---+



In [23]:
#2.8	Inner, left, right, full joins
data1 = [
    (1, "Alice", "HR"),
    (2, "Bob", "IT"),
    (3, "Cathy", "Finance"),
    (4, "David", "IT")
]
columns1 = ["emp_id", "name", "dept"]

df1 = spark.createDataFrame(data1, columns1)
df1.show()

data2 = [
    (1, 5000),
    (2, 6000),
    (4, 7000),
    (5, 8000)
]
columns2 = ["emp_id", "salary"]

df2 = spark.createDataFrame(data2, columns2)
df2.show()

print("INNER JOIN\n")
inner_join_df = df1.join(df2, on = "emp_id", how = "inner")
inner_join_df.show()

print("LEFT JOIN\n")
df1.join(df2, on = "emp_id", how = "left").show()

print("RIGHT JOIN\n")
df1.join(df2, on = "emp_id", how = "right").show()

+------+-----+-------+
|emp_id| name|   dept|
+------+-----+-------+
|     1|Alice|     HR|
|     2|  Bob|     IT|
|     3|Cathy|Finance|
|     4|David|     IT|
+------+-----+-------+

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5|  8000|
+------+------+

INNER JOIN

+------+-----+----+------+
|emp_id| name|dept|salary|
+------+-----+----+------+
|     1|Alice|  HR|  5000|
|     2|  Bob|  IT|  6000|
|     4|David|  IT|  7000|
+------+-----+----+------+

LEFT JOIN

+------+-----+-------+------+
|emp_id| name|   dept|salary|
+------+-----+-------+------+
|     1|Alice|     HR|  5000|
|     2|  Bob|     IT|  6000|
|     3|Cathy|Finance|  NULL|
|     4|David|     IT|  7000|
+------+-----+-------+------+

RIGHT JOIN

+------+-----+----+------+
|emp_id| name|dept|salary|
+------+-----+----+------+
|     1|Alice|  HR|  5000|
|     2|  Bob|  IT|  6000|
|     5| NULL|NULL|  8000|
|     4|David|  IT|  7000|
+------+-----+----+------+



In [24]:
# 2.9	Semi and anti joins
print("SEMI JOIN\n")
df1.join(df2, on = "emp_id", how = "left_semi").show()
print("ANTI JOIN\n")
df1.join(df2, on = "emp_id", how = "left_anti").show()

#2.1	Self joins
print("SELF JOIN\n")
df1.join(df1, on = "emp_id", how = "inner").show()

SEMI JOIN

+------+-----+----+
|emp_id| name|dept|
+------+-----+----+
|     1|Alice|  HR|
|     2|  Bob|  IT|
|     4|David|  IT|
+------+-----+----+

ANTI JOIN

+------+-----+-------+
|emp_id| name|   dept|
+------+-----+-------+
|     3|Cathy|Finance|
+------+-----+-------+

SELF JOIN

+------+-----+-------+-----+-------+
|emp_id| name|   dept| name|   dept|
+------+-----+-------+-----+-------+
|     1|Alice|     HR|Alice|     HR|
|     2|  Bob|     IT|  Bob|     IT|
|     3|Cathy|Finance|Cathy|Finance|
|     4|David|     IT|David|     IT|
+------+-----+-------+-----+-------+



In [25]:
data1 = [
    (1, "Alice", "HR"),
    (2, "Bob", "IT"),
    (3, "Cathy", "Finance"),
    (4, "David", "IT"),
    (5, "Suther", "BYju")
]
columns1 = ["emp_id", "name", "dept"]

df1 = spark.createDataFrame(data1, columns1)
df1.show()

data2 = [
    (1, 5000),
    (2, 6000),
    (4, 7000),
    (5, 8000),
    (6, None)
]
columns2 = ["emp_id", "salary"]

df2 = spark.createDataFrame(data2, columns2)
df2.show()

+------+------+-------+
|emp_id|  name|   dept|
+------+------+-------+
|     1| Alice|     HR|
|     2|   Bob|     IT|
|     3| Cathy|Finance|
|     4| David|     IT|
|     5|Suther|   BYju|
+------+------+-------+

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5|  8000|
|     6|  NULL|
+------+------+



In [26]:
#2.11	Handling nulls (fillna, dropna, na.replace)
df2.fillna({"salary":10000}).show()
df2.dropna().show()
df2.fillna({"salary":10000}).na.replace(10000,20000).show()

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5|  8000|
|     6| 10000|
+------+------+

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5|  8000|
+------+------+

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5|  8000|
|     6| 20000|
+------+------+



In [27]:
from pyspark.sql.functions import equal_null
#2.12	Replace specific values in a column
df2.replace(8000,20000).show()

df2.na.replace({4:5},subset = ["salary"]).show()

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5| 20000|
|     6|  NULL|
+------+------+

+------+------+
|emp_id|salary|
+------+------+
|     1|  5000|
|     2|  6000|
|     4|  7000|
|     5|  8000|
|     6|  NULL|
+------+------+



In [28]:
#2.13	Cache and uncache data
df2.cache()
df2.unpersist()

DataFrame[emp_id: bigint, salary: bigint]

In [29]:
#2.14	Explain storage levels
df2.storageLevel

StorageLevel(False, False, False, False, 1)

In [30]:
# Sample data
data = [
    ("Alice", "Math", 95),
    ("Bob", "Math", 90),
    ("Charlie", "Math", 90),
    ("David", "Math", 85),
    ("Alice", "Physics", 88),
    ("Bob", "Physics", 92),
    ("Charlie", "Physics", 85)
]

columns = ["Name", "Subject", "Score"]

df = spark.createDataFrame(data, schema=columns)
df.show()

+-------+-------+-----+
|   Name|Subject|Score|
+-------+-------+-----+
|  Alice|   Math|   95|
|    Bob|   Math|   90|
|Charlie|   Math|   90|
|  David|   Math|   85|
|  Alice|Physics|   88|
|    Bob|Physics|   92|
|Charlie|Physics|   85|
+-------+-------+-----+



In [32]:
#3.1 Ranking functions (rank, dense_rank, row_number)
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank, row_number

windowSpec = Window.partitionBy("Subject").orderBy(df.Score.desc())

In [50]:
just_score_window = Window.orderBy(df.Score.desc())

df.withColumn("rank",rank().over(just_score_window)).show()
df.withColumn("dense_rank",dense_rank().over(Window.partitionBy(df.Subject).orderBy(df.Score.desc()))).show()
df.withColumn("row_number",row_number().over(windowSpec)).show()

+---------+-------+-----+----+
|full_name|Subject|Score|rank|
+---------+-------+-----+----+
|    Alice|   Math|   95|   1|
|      Bob|Physics|   92|   2|
|      Bob|   Math|   90|   3|
|  Charlie|   Math|   90|   3|
|    Alice|Physics|   88|   5|
|    David|   Math|   85|   6|
|  Charlie|Physics|   85|   6|
+---------+-------+-----+----+

+---------+-------+-----+----------+
|full_name|Subject|Score|dense_rank|
+---------+-------+-----+----------+
|    Alice|   Math|   95|         1|
|      Bob|   Math|   90|         2|
|  Charlie|   Math|   90|         2|
|    David|   Math|   85|         3|
|      Bob|Physics|   92|         1|
|    Alice|Physics|   88|         2|
|  Charlie|Physics|   85|         3|
+---------+-------+-----+----------+

+---------+-------+-----+----------+
|full_name|Subject|Score|row_number|
+---------+-------+-----+----------+
|    Alice|   Math|   95|         1|
|      Bob|   Math|   90|         2|
|  Charlie|   Math|   90|         3|
|    David|   Math|   85|   

In [51]:
# 3.2	Aggregations over a window (lead, lag, sum over partition)
from pyspark.sql.functions import  lag, lead
df.withColumn("prev_score", lag("Score").over(windowSpec)).show()
df.withColumn("next_score", lead("Score").over(windowSpec)).show()

+---------+-------+-----+----------+
|full_name|Subject|Score|prev_score|
+---------+-------+-----+----------+
|    Alice|   Math|   95|      NULL|
|      Bob|   Math|   90|        95|
|  Charlie|   Math|   90|        90|
|    David|   Math|   85|        90|
|      Bob|Physics|   92|      NULL|
|    Alice|Physics|   88|        92|
|  Charlie|Physics|   85|        88|
+---------+-------+-----+----------+

+---------+-------+-----+----------+
|full_name|Subject|Score|next_score|
+---------+-------+-----+----------+
|    Alice|   Math|   95|        90|
|      Bob|   Math|   90|        90|
|  Charlie|   Math|   90|        85|
|    David|   Math|   85|      NULL|
|      Bob|Physics|   92|        88|
|    Alice|Physics|   88|        85|
|  Charlie|Physics|   85|      NULL|
+---------+-------+-----+----------+



In [52]:
#3.3	Broadcast joins for small datasets
from pyspark.sql.functions import broadcast
df1.join(broadcast(df2), on = "emp_id", how = "inner").show()

+------+------+----+------+
|emp_id|  name|dept|salary|
+------+------+----+------+
|     1| Alice|  HR|  5000|
|     2|   Bob|  IT|  6000|
|     4| David|  IT|  7000|
|     5|Suther|BYju|  8000|
+------+------+----+------+



In [53]:
#3.4	Cross joins
df1.crossJoin(df2).show()

+------+------+-------+------+------+
|emp_id|  name|   dept|emp_id|salary|
+------+------+-------+------+------+
|     1| Alice|     HR|     1|  5000|
|     1| Alice|     HR|     2|  6000|
|     2|   Bob|     IT|     1|  5000|
|     2|   Bob|     IT|     2|  6000|
|     1| Alice|     HR|     4|  7000|
|     1| Alice|     HR|     5|  8000|
|     1| Alice|     HR|     6|  NULL|
|     2|   Bob|     IT|     4|  7000|
|     2|   Bob|     IT|     5|  8000|
|     2|   Bob|     IT|     6|  NULL|
|     3| Cathy|Finance|     1|  5000|
|     3| Cathy|Finance|     2|  6000|
|     4| David|     IT|     1|  5000|
|     4| David|     IT|     2|  6000|
|     5|Suther|   BYju|     1|  5000|
|     5|Suther|   BYju|     2|  6000|
|     3| Cathy|Finance|     4|  7000|
|     3| Cathy|Finance|     5|  8000|
|     3| Cathy|Finance|     6|  NULL|
|     4| David|     IT|     4|  7000|
+------+------+-------+------+------+
only showing top 20 rows



In [54]:
df.show()

+---------+-------+-----+
|full_name|Subject|Score|
+---------+-------+-----+
|    Alice|   Math|   95|
|      Bob|   Math|   90|
|  Charlie|   Math|   90|
|    David|   Math|   85|
|    Alice|Physics|   88|
|      Bob|Physics|   92|
|  Charlie|Physics|   85|
+---------+-------+-----+



In [55]:
#3.5	Create and register Python UDFs
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

double_udf = udf(lambda x:x*2, IntegerType())
df.withColumn("newCol",double_udf("Score")).show()

+---------+-------+-----+------+
|full_name|Subject|Score|newCol|
+---------+-------+-----+------+
|    Alice|   Math|   95|   190|
|      Bob|   Math|   90|   180|
|  Charlie|   Math|   90|   180|
|    David|   Math|   85|   170|
|    Alice|Physics|   88|   176|
|      Bob|Physics|   92|   184|
|  Charlie|Physics|   85|   170|
+---------+-------+-----+------+



In [57]:
#3.7	Explode arrays and maps
from pyspark.sql.functions import explode
data = [
    (1, ["a", "b", "c"]),
    (2, ["x", "y"])
]
df = spark.createDataFrame(data, ["id", "letters"])
df.show()
df_exploded = df.withColumn("letter", explode("letters"))
df_exploded.show()

+---+---------+
| id|  letters|
+---+---------+
|  1|[a, b, c]|
|  2|   [x, y]|
+---+---------+

+---+---------+------+
| id|  letters|letter|
+---+---------+------+
|  1|[a, b, c]|     a|
|  1|[a, b, c]|     b|
|  1|[a, b, c]|     c|
|  2|   [x, y]|     x|
|  2|   [x, y]|     y|
+---+---------+------+



In [66]:
data = [
    (1, {"a": 10, "b": 20}),
    (2, {"x": 5})
]
df = spark.createDataFrame(data, ["id", "scores"])
df.show()
df_exploded = df.withColumn(("key","value"), explode("scores"))
df_exploded.show()

+---+------------------+
| id|            scores|
+---+------------------+
|  1|{a -> 10, b -> 20}|
|  2|          {x -> 5}|
+---+------------------+



Py4JError: An error occurred while calling o1047.withColumn. Trace:
py4j.Py4JException: Method withColumn([class java.util.ArrayList, class org.apache.spark.sql.Column]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [68]:
#3.1	Write data with partitioning & bucketing
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("salary", IntegerType(), True)
])

# Sample data
data = [
    (1, "Alice", "HR", 5000),
    (2, "Bob", "IT", 6000),
    (3, "Charlie", "IT", 7000),
    (4, "David", "HR", 5500),
    (5, "Eve", "Finance", 6500),
    (6, "Frank", "Finance", 6000),
]

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.show()

+---+-------+----------+------+
| id|   name|department|salary|
+---+-------+----------+------+
|  1|  Alice|        HR|  5000|
|  2|    Bob|        IT|  6000|
|  3|Charlie|        IT|  7000|
|  4|  David|        HR|  5500|
|  5|    Eve|   Finance|  6500|
|  6|  Frank|   Finance|  6000|
+---+-------+----------+------+



In [70]:
df.write.mode("overwrite").partitionBy("department").parquet("/content/employee_partitioned")

In [72]:
df.write.mode("overwrite") \
    .bucketBy(3, "department") \
    .sortBy("salary") \
    .saveAsTable("employee_bucketed")

In [73]:
df.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[id#1329,name#1330,department#1331,salary#1332]


